In [3]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('C:/Users/timei/Documents/Datalogi/Semester6/Bachelor/pm4py-dcr') # working directory should be pm4py-dcr (the one behind notebooks)
print(os.getcwd())

C:\Users\timei\Documents\Datalogi\Semester6\Bachelor\pm4py-dcr\notebooks
C:\Users\timei\Documents\Datalogi\Semester6\Bachelor\pm4py-dcr


In [4]:
import pm4py
from copy import deepcopy

In [5]:
from pm4py.objects.dcr import semantics as dcr_semantics
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.algo.discovery.dcr_discover import algorithm as dcr_alg
from pm4py.objects.dcr import enforcement

In [30]:
dcr = {
    'events': {'release', 'archive', 'delete', 'readmit', 'unarchive'},
    'labels': set(),
    'labelMapping': set(),
    'conditionsFor': {'delete': {'release'}, 'unarchive':{'release'}, 'archive':{'release'}},  # this should be a dict with events as keys and sets as values
    'milestonesFor': {'delete': {'archive'}},
    'responseTo': {},
    'noResponseTo': {},
    'includesTo': {'release':{'archive', 'delete', 'unarchive'}},
    'excludesTo': {'readmit':{'unarchive', 'archive', 'delete'}},
    'conditionsForDelays': {'unarchive':{('archive', 2920)}},  # this should be a dict with events as keys and tuples as values
    'responseToDeadlines': {'release':{('archive',14), ('delete',14)}},
    'marking': {'executed': set(),
                'included': {'release', 'archive', 'delete', 'readmit', 'unarchive'},
                'pending': set(),
                'executedTime': {}, # Gives the time since a event was executed
                'pendingDeadline': {} # The deadline until an event must be executed 
                }
}

In [ ]:
dcr_semantics.enabled(dcr)

In [9]:
dcr_semantics.is_accepting(dcr)

True

In [212]:
event_to_execute = 'release' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event
print(dcr_semantics.enabled(dcr)) # now these are the newly enabled events

{'release', 'archive', 'readmit'}


In [11]:
event_to_execute = 'hallo' # update this event in order to simulate the execution of the event
dcr_semantics.execute(event_to_execute, dcr) # here you execute the event

False

In [139]:
test = {'hej': 14, 'hello':13}
print(test)
test["hi"] = (25)
print(test)
test['hi'] = 10
print(test)

{'hej': 14, 'hello': 13}
{'hej': 14, 'hello': 13, 'hi': 25}
{'hej': 14, 'hello': 13, 'hi': 10}


In [7]:
g = enforcement.IGraph()
g.from_dcr_to_graph(dcr)

In [51]:
e = enforcement.Enforcement_mechanisme(dcr)

In [28]:
def check_for_responses(graph):
        for e in graph:
            for e_prime in graph[e]:
                if e_prime in dcr['responseToDeadlines']:
                    if e in dcr['responseToDeadlines'][e_prime]:
                        return True
                if e_prime in dcr['responseTo']:
                    if e in dcr['responseTo'][e_prime]:
                        return True
                if e_prime in dcr['includesTo']:
                    if e in dcr['includesTo'][e_prime]:
                        return True
        return False

In [31]:
print(check_for_response_and_delay(g.graph))

False
